In [ ]:
import asyncio

from request.request import Request

vdc_request = Request()

asyncio.create_task(vdc_request.start())


In [ ]:
online_players = asyncio.create_task(vdc_request.get_online_player_response())
avicia = asyncio.create_task(vdc_request.get_guild_response("Avicia", False))
lindafelix = asyncio.create_task(vdc_request.get_player_response("lindafelix"))
fazuh = asyncio.create_task(vdc_request.get_player_response("FAZuH"))
marcus = asyncio.create_task(vdc_request.get_player_response("Maarcus1"))

In [ ]:
from pprint import pprint
from datetime import datetime

In [ ]:
online_player_resp = online_players.result()
lindafelix_resp = lindafelix.result()
avicia_resp = avicia.result()

fazuh_resp = fazuh.result()
marcus_resp = marcus.result()

fazuh_content = asyncio.create_task(fazuh_resp.json())
marcus_content = asyncio.create_task(marcus_resp.json())

In [ ]:
fazuh_content = fazuh_content.result()
marcus_content = marcus_content.result()

In [ ]:
head_dict = {k: v for k, v in online_player_resp.headers.items()}
pprint(head_dict)
expire_dt = datetime.strptime(head_dict["Expires"], '%a, %d %b %Y %H:%M:%S %Z')

In [ ]:
import mysql.connector

In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root"
)
cursor = db.cursor()

In [ ]:
cursor.close()

In [ ]:
pprint(marcus_content)

In [ ]:
marcus_ch_uuids = [e for e in marcus_content["characters"].keys()]

In [ ]:
xps = []
blockss = []
items_idd = []
for chuuid in marcus_ch_uuids:
    xps.append(marcus_content["characters"][chuuid]["xp"])
    blockss.append(marcus_content["characters"][chuuid]["blocksWalked"])
    items_idd.append(marcus_content["characters"][chuuid]["itemsIdentified"])
xps.sort()
blockss.sort()

In [ ]:
for e in items_idd:
    print(f"{e:,}")

In [33]:
from time import time
print(f"{time():,}")

1,698,003,319.7769663
